In [1]:
import numpy as np
import time

import ipywidgets
from IPython.display import display
from jetcam.usb_camera import USBCamera

import jetson.inference
import jetson.utils
from jetcam.utils import bgr8_to_jpeg
import cv2
import paho.mqtt.client as mqtt
from IPython.display import clear_output

In [2]:
def send_data(final_string):
    global products_list, out
    client = mqtt.Client()
    client.connect("192.168.136.253", 1883, 60)
    print("connected")
    client.publish("test", final_string)
    client.disconnect()
    products_list = []
    with out:
        clear_output()

In [3]:
available_items = [
    "apple", "backpack", "banana", "bed", "blender", "book", "bottle",
    "bowl", "broccoli", "cake", "carrot", "cell phone", "chair", "clock", "couch", "cup",
    "desk", "dining table", "donut", "door", "fork", "hair drier", "handbag", "hat", "keyboard", "knife",
    "laptop", "microwave", "mirror", "mouse", "orange", "oven", "pizza", "plate", "potted plant", "refrigerator",
    "remote", "sandwich", "scissors", "shoe", "sink", "spoon", "suitcase", "teddy bear", "toaster", "tooth brush", "tv", "umbrella", "vase", "window"
]

In [4]:
# Creating UI widgets

name_widget = ipywidgets.Text(description='Name', value="First Last")
phone_widget = ipywidgets.IntText(description= "phone number", value=9030116001)

scan_widget = ipywidgets.Button(description="Scan")
checkout_widget = ipywidgets.Button(description="Checkout")

first_layout = ipywidgets.VBox([ name_widget, phone_widget])
second_layout = ipywidgets.VBox([ scan_widget, checkout_widget ])

top = ipywidgets.HBox([ first_layout, second_layout])

image_widget = ipywidgets.Image(format='jpeg')
# image_widget.value = bgr8_to_jpeg(img)

out = ipywidgets.Output()

live_execution_widget = ipywidgets.VBox([
    top,
    #image_widget,
    out
])

products_list = []

def scan_clicked(wdgt):
    
    global product
    global products_list 
    
    if product != "" and product in available_items: 
        product = product.replace(" ", "")
        products_list.append(product)      
        with out:
            print(product)

        
scan_widget.on_click(scan_clicked)

def checkout_clicked(wdgt):
    with out:
        print("Checking out")
    final_string = name_widget.value + " " + str(phone_widget.value) + " " + ",".join(products_list)
    send_data(final_string)
    # products_list = []
    # with out:
        # clear_output()

checkout_widget.on_click(checkout_clicked)

display(image_widget)
display(live_execution_widget)

Image(value=b'', format='jpeg')

In [5]:
final_string = name_widget.value + " " + str(phone_widget.value) + " " + ",".join(products_list)
final_string

'First Last 9030116001 '

In [6]:
net = jetson.inference.detectNet()
camera = USBCamera(width=224, height=224, capture_width=640, capture_height=480, capture_device=0) 
img = camera.read()
camera.running = True 


product = ""


def update_image(change):
    global product 
    global net 
    img = change["new"]
    cimg = jetson.utils.cudaFromNumpy(img)

    detections = net.Detect(cimg, overlay='None')
    for detection in detections:
        product = net.GetClassDesc(detection.ClassID)
        if product in available_items:
            shape_a = (round(detection.Left), round(detection.Bottom))
            shape_b = (round(detection.Right), round(detection.Top))
            rgb_array = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            box_color = (0, 255, 255)
            cv2.rectangle(rgb_array, shape_a, shape_b, box_color, 4)
            stripped_label = product.rstrip('\n')
            cv2.putText(rgb_array, stripped_label,
                        (round(detection.Left) + 20, round(detection.Top) + 40),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,  # font scale
                        (255, 0, 0),
                        2)  # line type
            # convert back to BGR
            img = cv2.cvtColor(rgb_array, cv2.COLOR_RGB2BGR)
    
    img = cv2.resize(img, (640,480))
    image_widget.value = bgr8_to_jpeg(img)

In [7]:
camera.observe(update_image, names="value")

In [8]:
# camera.unobserve(update_image, names='value')